In [1]:
# Add rebel component https://github.com/Babelscape/rebel/blob/main/spacy_component.py
import spacy
import crosslingual_coreference
import requests
import re
import hashlib
import pandas as pd
from spacy import Language
from typing import List
from spacy.tokens import Doc, Span
from transformers import pipeline

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\GCM\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
def call_wiki_api(item):
    try:
        url = f"https://www.wikidata.org/w/api.php?action=wbsearchentities&search={item}&language=en&format=json"
        data = requests.get(url).json()
        # Return the first id (Could upgrade this in the future)
        return data['search'][0]['id']
    except:
        return 'id-less'

def extract_triplets(text):
    """
    Function to parse the generated text and extract the triplets
    """
    triplets = []
    relation, subject, relation, object_ = '', '', '', ''
    text = text.strip()
    current = 'x'
    for token in text.replace("<s>", "").replace("<pad>", "").replace("</s>", "").split():
        if token == "<triplet>":
            current = 't'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
                relation = ''
            subject = ''
        elif token == "<subj>":
            current = 's'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
            object_ = ''
        elif token == "<obj>":
            current = 'o'
            relation = ''
        else:
            if current == 't':
                subject += ' ' + token
            elif current == 's':
                object_ += ' ' + token
            elif current == 'o':
                relation += ' ' + token
    if subject != '' and relation != '' and object_ != '':
        triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})

    return triplets


@Language.factory(
    "rebel",
    requires=["doc.sents"],
    assigns=["doc._.rel"],
    default_config={
        "model_name": "Babelscape/rebel-large",
        "device": 0,
    },
)
class RebelComponent:
    def __init__(
        self,
        nlp,
        name,
        model_name: str,
        device: int,
    ):
        assert model_name is not None, ""
        self.triplet_extractor = pipeline("text2text-generation", model=model_name, tokenizer=model_name, device=device)
        self.entity_mapping = {}
        # Register custom extension on the Doc
        if not Doc.has_extension("rel"):
            Doc.set_extension("rel", default={})

    def get_wiki_id(self, item: str):
        mapping = self.entity_mapping.get(item)
        if mapping:
            return mapping
        else:
            res = call_wiki_api(item)
            self.entity_mapping[item] = res
            return res

    
    def _generate_triplets(self, sent: Span) -> List[dict]:
        output_ids = self.triplet_extractor(sent.text, return_tensors=True, return_text=False)[0]["generated_token_ids"]["output_ids"]
        extracted_text = self.triplet_extractor.tokenizer.batch_decode(output_ids[0])
        extracted_triplets = extract_triplets(extracted_text[0])
        return extracted_triplets

    def set_annotations(self, doc: Doc, triplets: List[dict]):
        for triplet in triplets:

            # Remove self-loops (relationships that start and end at the entity)
            if triplet['head'] == triplet['tail']:
                continue

            # Use regex to search for entities
            head_span = re.search(triplet["head"], doc.text)
            tail_span = re.search(triplet["tail"], doc.text)

            # Skip the relation if both head and tail entities are not present in the text
            # Sometimes the Rebel model hallucinates some entities
            if not head_span or not tail_span:
                continue

            index = hashlib.sha1("".join([triplet['head'], triplet['tail'], triplet['type']]).encode('utf-8')).hexdigest()
            if index not in doc._.rel:
                # Get wiki ids and store results
                doc._.rel[index] = {"relation": triplet["type"], "head_span": {'text': triplet['head'], 'id': self.get_wiki_id(triplet['head'])}, "tail_span": {'text': triplet['tail'], 'id': self.get_wiki_id(triplet['tail'])}}

    def __call__(self, doc: Doc) -> Doc:
        for sent in doc.sents:
            sentence_triplets = self._generate_triplets(sent)
            self.set_annotations(doc, sentence_triplets)
        return doc

In [3]:
DEVICE = 0 # Number of the GPU, -1 if want to use CPU

# Add coreference resolution model
coref = spacy.load('en_core_web_sm', disable=['ner', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer'])
coref.add_pipe("xx_coref", 
    config={"chunk_size": 2500, 
            "chunk_overlap": 2, 
            "device": DEVICE}
              )

# Define rel extraction model
rel_ext = spacy.load('en_core_web_sm', disable=['ner', 'lemmatizer', 'attribute_rules', 'tagger'])
rel_ext.add_pipe("rebel", 
                 config={
                     'device':DEVICE,
                     'model_name':'Babelscape/rebel-large'} # Model used, will default to 'Babelscape/rebel-large' if not given
                )

error loading _jsonnet (this is expected on Windows), treating C:\Users\GCM\AppData\Local\Temp\tmpbcmi9qxu\config.json as plain json
Some weights of the model checkpoint at nreimers/mMiniLMv2-L12-H384-distilled-from-XLMR-Large were not used when initializing XLMRobertaModel: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaModel were not initialized from the model checkpoint at nreimers/mMiniLMv2-L12-H384-distilled-from-XLMR-Large and 

In [4]:
# crosslingual_coreference implementation
def coref_res(text_series):
    coref_text_series = text_series.apply(lambda x : coref(x)._.resolved_text)
    return(coref_text_series)

# # choose minilm for speed/memory and info_xlm for accuracy
# predictor = Predictor(
#     language="en_core_web_sm", device=-1, model_name="minilm"
# )

In [5]:
def link_entities_text(text):
    try:
        ent_rel_lst = list(rel_ext(text)._.rel.values())
    except:
        print("Could not extract relationships for text")
        ent_rel_lst = [{'relation': 'rel_err',
                        'head_span': {'text': 'rel_err', 'id': 'rel_err'},
                        'tail_span': {'text': 'rel_err', 'id': 'rel_err'}}]
        
    entity_df = pd.DataFrame()
    rel_lst = []
    head_text_lst = []
    head_wiki_id_lst = []
    tail_text_lst = []
    tail_wiki_id_lst = []
    for i in range(len(ent_rel_lst)):
        rel_lst.append(ent_rel_lst[i]['relation'])
        head_text_lst.append(ent_rel_lst[i]['head_span']['text'])
        head_wiki_id_lst.append(ent_rel_lst[i]['head_span']['id'])
        tail_text_lst.append(ent_rel_lst[i]['tail_span']['text'])
        tail_wiki_id_lst.append(ent_rel_lst[i]['tail_span']['id'])
    entity_df['head_text'] = head_text_lst
    entity_df['head_wiki_id'] = head_wiki_id_lst
    entity_df['relation'] = rel_lst
    entity_df['tail_text'] = tail_text_lst
    entity_df['tail_wiki_id'] = tail_wiki_id_lst
    return(entity_df)

def link_entities(text_series):
    entity_df_series = text_series.apply(lambda x : link_entities_text(x))
    return(entity_df_series)

In [6]:
data_df = pd.read_csv('E:\\GIT_REPOS\\LAB\\Literature_summary\\Test\\Entity_edgelist\\Input\\entomology-machine-learning-csv.csv')
# data_df = data_df.drop('File', axis=1)
data_df = data_df.drop_duplicates().reset_index(drop=True)
data_df

,Authors,Author(s) ID,Title,Year,Source title,Volume,Issue,Art. No.,Page start,Page end,...,ISBN,CODEN,PubMed ID,Language of Original Document,Abbreviated Source Title,Document Type,Publication Stage,Open Access,Source,EID
0,"MOORE D., PENIKAS J., RANKIN M.A.",55197635600;6504171017;7006868718;,Regional specialization for an optomotor respo...,1981.0,Physiological Entomology,6,1,NaN,61,69,...,NaN,NaN,NaN,English,Physiol.Entomol.,Article,Final,NaN,Scopus,2-s2.0-84981620126
1,Hoyle G.,7006576967;,The scope of neuroethology,1984.0,Behavioral and Brain Sciences,7,3,NaN,367,381,...,NaN,NaN,NaN,English,Behav. Brain Sci.,Article,Final,NaN,Scopus,2-s2.0-84973935944
2,Skinner B.F.,7005596882;,Selection by consequences,1984.0,Behavioral and Brain Sciences,7,4,NaN,477,481,...,NaN,NaN,NaN,English,Behav. Brain Sci.,Article,Final,NaN,Scopus,2-s2.0-84971114253
3,"Menzel R., Greggers U.",7102000725;6506245498;,Natural phototaxis and its relationship to col...,1985.0,Journal of Comparative Physiology A,157,3,NaN,311,321,...,NaN,JCPAD,NaN,English,J. Comp. Physiol.,Article,Final,NaN,Scopus,2-s2.0-0006976113
4,"Brill J.H., Mayfield H.T., Mar T., Bertsch W.",7103055208;6603702494;24427642600;7004578956;,Use of computerized pattern recognition in the...,1985.0,Journal of Chromatography A,349,1,NaN,31,38,...,NaN,JCRAE,4086644.0,English,J. Chromatogr. A,Article,Final,NaN,Scopus,2-s2.0-0022420221
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3832,"Pian C., Mao S., Zhang G., Du J., Li F., Leung...",56181512400;57202392306;57189468170;5721520425...,Discovering Cancer-Related miRNAs from miRNA-T...,2020.0,Molecular Therapy - Nucleic Acids,19,NaN,NaN,1423,1433,...,NaN,NaN,NaN,English,Mol. Ther. Nucl. Acids,Article,Final,"All Open Access, Gold, Green",Scopus,2-s2.0-85080101579
3833,"Bleker J., Kwee T.C., Dierckx R.A.J.O., de Jon...",57212091754;16432912900;7102816622;7005938218;...,Multiparametric MRI and auto-fixed volume of i...,2020.0,European Radiology,30,3,NaN,1313,1324,...,NaN,EURAE,31776744.0,English,Eur. Radiol.,Article,Final,"All Open Access, Hybrid Gold, Green",Scopus,2-s2.0-85075888221
3834,"Jose V S., Nameer P.O.",57211856052;6507739308;,The expanding distribution of the Indian Peafo...,2020.0,Ecological Indicators,110,NaN,105930,NaN,NaN,...,NaN,NaN,NaN,English,Ecol. Indic.,Article,Final,NaN,Scopus,2-s2.0-85075189322
3835,"Kaszubinski S.F., Pechal J.L., Schmidt C.J., J...",57211628505;35272981100;35493831100;5698254900...,Evaluating Bioinformatic Pipeline Performance ...,2020.0,Journal of Forensic Sciences,65,2,NaN,513,525,...,NaN,JFSCA,31657871.0,English,J. Forensic Sci.,Article,Final,NaN,Scopus,2-s2.0-85074669672


In [7]:
# import data
# data_df = pd.read_csv('E:\GIT_REPOS\LAB\Literature_summary\TPN\Papers\\scopus.csv')
data_df = data_df[data_df["Abstract"] != '[No abstract available]']
data_df.reset_index(inplace=True, drop=True)
data_df["Abstract"] = data_df["Abstract"].str.replace(r'(', '')
data_df["Abstract"] = data_df["Abstract"].str.replace(r')', '')
data_df["Abstract"] = data_df["Abstract"].str.replace(r"'", '')
data_df["Abstract"] = data_df["Abstract"].str.replace(r"'", '')
data_df["Abstract"] = data_df["Abstract"].str.replace(r'"', '')
data_df["Abstract"] = data_df["Abstract"].str.replace(r'"', '')
data_df["Abstract"] = data_df["Abstract"].astype(str)

C:\Users\GCM\AppData\Local\Temp\ipykernel_25880\412532311.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data_df["Abstract"] = data_df["Abstract"].str.replace(r'(', '')
C:\Users\GCM\AppData\Local\Temp\ipykernel_25880\412532311.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data_df["Abstract"] = data_df["Abstract"].str.replace(r')', '')


In [8]:
win_size = 100
start_point = 0 # default to 0
# coref_lst = []
entities_df_lst = []
for i in range(start_point, len(data_df), win_size):
    coref_series = coref_res(text_series=data_df["Abstract"].iloc[i:i+win_size])
    print('coref done', i)
    link_entities_series = link_entities(text_series=coref_series)
    print('entity linking done', i)
    entities_df = pd.concat(link_entities_series.tolist())
    print('df create done', i)
    entities_df_lst.append(entities_df)
    print('df to list done', i, '\n')
all_entities_df = pd.concat(entities_df_lst)
all_entities_df.reset_index(drop=True, inplace=True)
edge_lst_df = all_entities_df.value_counts().reset_index().rename(columns={0: "count"})
edge_lst_df.to_csv('entity_weighted_edgelist_entemology_ML.csv')

coref done 0
entity linking done 0
df create done 0
df to list done 0 

coref done 100
entity linking done 100
df create done 100
df to list done 100 

coref done 200
entity linking done 200
df create done 200
df to list done 200 

coref done 300
entity linking done 300
df create done 300
df to list done 300 

coref done 400
entity linking done 400
df create done 400
df to list done 400 

coref done 500
entity linking done 500
df create done 500
df to list done 500 

coref done 600
entity linking done 600
df create done 600
df to list done 600 

coref done 700
entity linking done 700
df create done 700
df to list done 700 

coref done 800
entity linking done 800
df create done 800
df to list done 800 

coref done 900
entity linking done 900
df create done 900
df to list done 900 

coref done 1000
entity linking done 1000
df create done 1000
df to list done 1000 

coref done 1100


C:\Users\GCM\AppData\Local\Temp\ipykernel_25880\4235504974.py:94: FutureWarning: Possible nested set at position 6
  head_span = re.search(triplet["head"], doc.text)


Could not extract relationships for text
entity linking done 1100
df create done 1100
df to list done 1100 

coref done 1200
Could not extract relationships for text
entity linking done 1200
df create done 1200
df to list done 1200 

coref done 1300
entity linking done 1300
df create done 1300
df to list done 1300 

coref done 1400
entity linking done 1400
df create done 1400
df to list done 1400 

coref done 1500
entity linking done 1500
df create done 1500
df to list done 1500 

coref done 1600
entity linking done 1600
df create done 1600
df to list done 1600 

coref done 1700
entity linking done 1700
df create done 1700
df to list done 1700 

coref done 1800
entity linking done 1800
df create done 1800
df to list done 1800 

coref done 1900
entity linking done 1900
df create done 1900
df to list done 1900 

coref done 2000
entity linking done 2000
df create done 2000
df to list done 2000 

coref done 2100
Could not extract relationships for text
entity linking done 2100
df create do

IndexError: list index out of range

In [ ]:
# coref_series = coref_res(text_series=data_df["Abstract"][:1000])
# link_entities_series = link_entities(text_series=coref_series)
# all_entities_df = pd.concat(link_entities_series.tolist())